In [ ]:
import whisperx
import csv
import json

In [ ]:
device = "cuda" 
audio_file = "~/test.wav"
batch_size = 5 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

#Fix for TypeError: TranscriptionOptions.__new__()
options = {
        "max_new_tokens": None,
        "clip_timestamps": None,
        "hallucination_silence_threshold": None,
    }

In [ ]:
model = whisperx.load_model("large-v3", device, compute_type=compute_type, language='nl', asr_options=options)

In [ ]:
audio = whisperx.load_audio(audio_file)
result = model.transcribe(audio, batch_size=batch_size)

In [ ]:
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

In [ ]:
diarize_model = whisperx.DiarizationPipeline(use_auth_token='', device=device)

In [ ]:
diarize_segments = diarize_model(audio, max_speakers=2)

In [ ]:
result = whisperx.assign_word_speakers(diarize_segments, result)

In [ ]:
def dict_to_csv(data, filename):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['', 'start', 'end', 'text', 'words', 'speaker'])

        for row_num, segment in enumerate(data['segments'], start=1):
            words_json = json.dumps(segment['words'])
            writer.writerow([row_num, segment['start'], segment['end'], segment['text'], words_json, segment['speaker']])

In [ ]:
dict_to_csv(data=result, filename="~/test.csv")